<a href="https://colab.research.google.com/github/MrPablunt/organizador_beats/blob/main/organizador_beats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# -*- coding: utf-8 -*-
import gspread
from google.oauth2.service_account import Credentials
import os
import librosa
import soundfile as sf
from mutagen.mp3 import MP3
from mutagen.id3 import ID3NoHeaderError, ID3, TIT2, TPE1, TCON, TXXX
import requests
from googleapiclient.discovery import build
import datetime

# --- CONFIGURACIÓN CRÍTICA: ¡VERIFICAR CADA VALOR! ---
# Ruta al archivo JSON de tus credenciales de Google Cloud
SERVICE_ACCOUNT_FILE = 'braided-grammar-465202-t4-f058b52a6154.json'

# --- Archivo de configuración para el SPREADSHEET_ID ---
# Este archivo guardará el ID de la Google Sheet. Se creará si no existe.
CONFIG_FILE = 'config.txt'

# ID de tu Google Sheet 'Catálogo de Beats' (ID inicial para la primera vez o si config.txt está vacío)
# El script intentará usar el ID de CONFIG_FILE primero. Si no lo encuentra, usará este ID.
# Si aún no lo encuentra o no existe, intentará crear una nueva hoja y actualizará CONFIG_FILE.
# ¡IMPORTANTE! Este SPREADSHEET_ID hardcodeado solo se usará si config.txt está vacío o falla.
# Una vez que config.txt se escriba, siempre leerá de ahí.
SPREADSHEET_ID = '19KvhoVYV0XIRZ3FfFTwBRPXjNSMyeAkFYUFl6I1kUEQ'
# Nombre EXACTO de la pestaña/hoja dentro de tu Google Sheet. Normalmente es 'Hoja 1'.
WORKSHEET_NAME = 'Hoja 1'

# ID de tu carpeta 'Nuevos Beats' en Google Drive (donde subes los beats a organizar)
NUEVOS_BEATS_FOLDER_ID = '1fwLKWkWvn8TY6SSL7kQgaVtZ5olZFuyw'

# ID de tu carpeta 'Beats organizados por género' en Google Drive (donde se crearán las subcarpetas de género)
ORGANIZED_BEATS_PARENT_FOLDER_ID = '1nILKS4_YglgyLRuA_X6107N6ThMbsB8o'

# --- CONFIGURACIÓN DE RUTAS LOCALES Y PERMISOS ---
# Alcances (permisos) que tu robot necesita en Google
SCOPE = [
    'https://www.googleapis.com/auth/spreadsheets.readonly',
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive',
    'https://www.googleapis.com/auth/drive.file'
]

# Carpeta temporal en tu computadora para descargar y procesar beats
DOWNLOAD_TEMP_DIR = 'temp_beats_processing'
if not os.path.exists(DOWNLOAD_TEMP_DIR):
    try:
        os.makedirs(DOWNLOAD_TEMP_DIR)
    except OSError as e:
        print(f"ERROR FATAL: No se pudo crear la carpeta temporal '{DOWNLOAD_TEMP_DIR}'. Mensaje: {e}")
        print("VERIFICAR: Permisos de escritura en el escritorio o ruta del script.")
        exit(1)

# --- FIN DE CONFIGURACIÓN Y DECLARACIONES ---

# --- FUNCIONES DE MANEJO DE ARCHIVO DE CONFIGURACIÓN ---
def read_spreadsheet_id_from_config():
    """Lee el ID de la hoja de cálculo desde el archivo de configuración."""
    if os.path.exists(CONFIG_FILE):
        try:
            with open(CONFIG_FILE, 'r', encoding='utf-8') as f:
                return f.readline().strip()
        except Exception as e:
            print(f"ADVERTENCIA: No se pudo leer el ID de la hoja desde '{CONFIG_FILE}'. Mensaje: {e}")
    return None

def write_spreadsheet_id_to_config(new_id):
    """Escribe el ID de la hoja de cálculo en el archivo de configuración."""
    try:
        with open(CONFIG_FILE, 'w', encoding='utf-8') as f:
            f.write(new_id + '\n')
        print(f"ACTUALIZACIÓN: Nuevo SPREADSHEET_ID '{new_id}' guardado en '{CONFIG_FILE}'.")
    except Exception as e:
        print(f"ERROR: No se pudo guardar el nuevo SPREADSHEET_ID en '{CONFIG_FILE}'. Mensaje: {e}")
        print("VERIFICAR: Permisos de escritura en la carpeta del script para crear/actualizar '{CONFIG_FILE}'.")
# --- FIN FUNCIONES DE MANEJO DE ARCHIVO DE CONFIGURACIÓN ---


def authenticate_google():
    """
    Autentica con Google usando la cuenta de servicio.
    Verifica que el archivo de credenciales exista y que los permisos sean correctos.
    """
    if not os.path.exists(SERVICE_ACCOUNT_FILE):
        print(f"ERROR CRÍTICO DE AUTENTICACIÓN: El archivo de credenciales '{SERVICE_ACCOUNT_FILE}' NO SE ENCONTRÓ.")
        print("POSIBLE SOLUCIÓN: ")
        print("  1. Verifica que el nombre del archivo en 'SERVICE_ACCOUNT_FILE' es EXACTO (incluyendo .json).")
        print("  2. Asegúrate de que el archivo JSON está en la MISMA CARPETA que este script.")
        print("  3. Si lo guardaste en otro lugar, pon la RUTA COMPLETA al archivo.")
        return None, None, None

    try:
        gc = gspread.service_account(filename=SERVICE_ACCOUNT_FILE)
        creds = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPE)
        drive_service = build('drive', 'v3', credentials=creds)

        print("Autenticación exitosa (gspread y Drive API).")
        return gc, drive_service, creds
    except Exception as e:
        print(f"ERROR CRÍTICO DE AUTENTICACIÓN: Falló la conexión con Google. Mensaje: {e}")
        print("POSIBLE SOLUCIÓN: ")
        print("  1. El archivo JSON puede estar corrupto o incorrecto.")
        print("  2. La CUENTA DE SERVICIO (el email que termina en @gserviceaccount.com) NO TIENE PERMISOS de 'Editor' en:")
        print("     - La Google Sheet (cuyo ID está en config.txt)")
        print(f"     - La carpeta 'Nuevos Beats' (ID: {NUEVOS_BEATS_FOLDER_ID})")
        print(f"     - La carpeta 'Beats organizados por género' (ID: {ORGANIZED_BEATS_PARENT_FOLDER_ID})")
        print("  3. Las APIs de 'Google Drive API' y 'Google Sheets API' NO están HABILITADAS en tu proyecto de Google Cloud.")
        return None, None, None

def get_sheet_data(gc, initial_spreadsheet_id, worksheet_name):
    """
    Intenta abrir la hoja de cálculo por ID. Si no la encuentra, intenta por nombre.
    Si aún no la encuentra, crea una nueva hoja. Actualiza el archivo de configuración con el ID utilizado.
    """
    spreadsheet = None
    spreadsheet_id_to_use = initial_spreadsheet_id # ID que intentaremos usar o que crearemos

    if spreadsheet_id_to_use:
        try:
            spreadsheet = gc.open_by_key(spreadsheet_id_to_use)
            print(f"CONEXIÓN: Hoja de cálculo '{spreadsheet.title}' abierta por ID: {spreadsheet.id}")
            write_spreadsheet_id_to_config(spreadsheet.id) # Guardar el ID confirmado en config.txt
            return spreadsheet, spreadsheet.get_all_records(), spreadsheet.row_values(1)
        except gspread.exceptions.SpreadsheetNotFound:
            print(f"ADVERTENCIA: Hoja de cálculo con ID '{spreadsheet_id_to_use}' NO ENCONTRADA o accesible.")
            print("Intentando buscar por nombre o crear una nueva...")
        except Exception as e:
            print(f"ERROR CRÍTICO: Falló al intentar abrir la hoja por ID '{spreadsheet_id_to_use}'. Mensaje: {e}")
            print("POSIBLE SOLUCIÓN: Podría ser un problema de permisos o de conexión a Google Sheets.")
            return None, None, None
    else:
        print("ADVERTENCIA: No se encontró un SPREADSHEET_ID inicial o en config.txt. Intentando buscar por nombre o crear una nueva hoja.")

    if spreadsheet is None: # Si no se pudo abrir por ID, intentar por nombre o crear
        try:
            print(f"Intentando abrir la hoja por nombre: '{worksheet_name}'...")
            spreadsheet = gc.open(worksheet_name)
            print(f"CONEXIÓN: Hoja de cálculo '{worksheet_name}' abierta por nombre: {spreadsheet.id}")
            write_spreadsheet_id_to_config(spreadsheet.id) # Guardar el ID del config.txt con el nombre encontrado
        except gspread.exceptions.SpreadsheetNotFound:
            print(f"ADVERTENCIA: Hoja de cálculo con nombre '{worksheet_name}' NO ENCONTRADA.")
            print("Creando una nueva hoja de cálculo...")
            try:
                timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
                new_sheet_name = f"Catalogo_Beats_AutoCreada_{timestamp}"
                spreadsheet = gc.create(new_sheet_name)
                print(f"¡ÉXITO! Nueva hoja '{new_sheet_name}' creada con ID: {spreadsheet.id}")
                write_spreadsheet_id_to_config(spreadsheet.id) # ¡Actualizar el archivo de configuración con el nuevo ID!
            except Exception as e:
                print(f"ERROR CRÍTICO: Falló la creación de una nueva hoja de cálculo. Mensaje: {e}")
                print("POSIBLE SOLUCIÓN: Permisos insuficientes para crear hojas (verificar rol de la cuenta de Google Cloud Console), o error grave de la API.")
                return None, None, None
        except Exception as e:
            print(f"ERROR CRÍTICO: No se pudo acceder a la hoja de cálculo por nombre '{worksheet_name}'. Mensaje: {e}")
            print("POSIBLE SOLUCIÓN: Podría ser un problema de permisos o de conexión a Google Sheets.")
            return None, None, None

    if spreadsheet is None:
        return None, None, None

    try:
        try:
            worksheet = spreadsheet.worksheet(worksheet_name)
        except gspread.exceptions.WorksheetNotFound:
            print(f"ADVERTENCIA: La hoja de trabajo (pestaña) '{worksheet_name}' no se encontró. Usando la primera hoja disponible.")
            worksheet = spreadsheet.get_worksheet(0)

        if not worksheet.row_values(1) or len(worksheet.row_values(1)) < len(['Nombre del Archivo Original', 'Género', 'Ruta en Drive (ID)', 'Enlace de Google Drive', 'BPM', 'Clave Armónica', 'Estado (PENDIENTE/ORGANIZADO)']):
             print("ADVERTENCIA: Hoja nueva o sin encabezados. Añadiendo encabezados necesarios.")
             header_row = ['Nombre del Archivo Original', 'Género', 'Ruta en Drive (ID)', 'Enlace de Google Drive', 'BPM', 'Clave Armónica', 'Estado (PENDIENTE/ORGANIZADO)']
             worksheet.update('A1', [header_row])

        return worksheet, worksheet.get_all_records(), worksheet.row_values(1)
    except Exception as e:
        print(f"ERROR CRÍTICO: No se pudo acceder a los datos de la hoja de cálculo. Mensaje: {e}")
        print("POSIBLE SOLUCIÓN: Verificar la estructura de la hoja o permisos de lectura.")
        return None, None, None


def update_sheet_row(worksheet, row_index, column_name, new_value, headers):
    """Actualiza una celda específica en la hoja de cálculo. Maneja errores de columna y API."""
    try:
        col_index = headers.index(column_name) + 1
        worksheet.update_cell(row_index, col_index, new_value)
    except ValueError:
        print(f"ADVERTENCIA: Columna '{column_name}' no encontrada en la hoja para la fila {row_index}. La información no se guardó para esta columna.")
        print("POSIBLE SOLUCIÓN: Verifica que el nombre de la columna en tu Google Sheet es EXACTO (incluyendo mayúsculas/minúsculas y espacios).")
    except Exception as e:
        print(f"ERROR: No se pudo actualizar la celda en la fila {row_index}, columna '{column_name}'. Mensaje: {e}")
        print("POSIBLE SOLUCIÓN: Podría ser un problema de conexión o cuota de la API de Google Sheets.")

def get_drive_file_info(drive_service, folder_id):
    """Obtiene información de los archivos en una carpeta de Google Drive. Maneja errores de acceso."""
    try:
        files_info = {}
        query = f"'{folder_id}' in parents and trashed = false"
        results = drive_service.files().list(q=query, fields="files(id, name, mimeType)").execute()
        items = results.get('files', [])
        for item in items:
            files_info[item['name']] = {'id': item['id'], 'mimeType': item['mimeType']}
        return files_info
    except Exception as e:
        print(f"ERROR: No se pudo acceder a la carpeta de Google Drive con ID '{folder_id}'. Mensaje: {e}")
        print("POSIBLE SOLUCIÓN: Verifica que el ID de la carpeta es CORRECTO y la CUENTA DE SERVICIO tiene permisos de 'Editor' en esa carpeta.")
        return {}

def download_file(drive_service, file_id, file_name, destination_folder, creds):
    """Descarga un archivo de Google Drive a una carpeta local. Maneja errores de descarga."""
    filepath = os.path.join(destination_folder, file_name)
    try:
        download_url = drive_service.files().get_media(fileId=file_id).uri
        headers = {'Authorization': f'Bearer {creds.token}'}
        response = requests.get(download_url, headers=headers, stream=True)
        response.raise_for_status()

        with open(filepath, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                if chunk:
                    f.write(chunk)
        return filepath
    except requests.exceptions.HTTPError as err:
        print(f"ERROR DE DESCARGA HTTP para '{file_name}' (ID: {file_id}). Código: {err.response.status_code}. Mensaje: {err.response.text}")
        print("POSIBLE SOLUCIÓN: El archivo no existe en Drive (ID incorrecto) o hay un problema temporal de Google Drive.")
        return None
    except requests.exceptions.ConnectionError as err:
        print(f"ERROR DE CONEXIÓN al descargar '{file_name}'. Mensaje: {err}")
        print("POSIBLE SOLUCIÓN: Problemas de conexión a internet o firewall bloqueando la conexión.")
        return None
    except Exception as e:
        print(f"ERROR GENERAL al descargar '{file_name}'. Mensaje: {e}")
        print("POSIBLE SOLUCIÓN: Verificar permisos de escritura en la carpeta temporal.")
        return None

def analyze_audio(filepath):
    """Analiza un archivo de audio para BPM y clave usando Librosa. Maneja errores de Librosa."""
    if not filepath:
        return "N/A", "N/A"
    try:
        y, sr = librosa.load(filepath, sr=None)

        onset_env = librosa.onset.onset_detect(y=y, sr=sr)
        tempo, _ = librosa.beat.beat_track(onset_env=onset_env, sr=sr)

        chroma = librosa.feature.chroma_stft(y=y, sr=sr)
        key_mode = librosa.key_to_notes(librosa.feature.tonnetz(y=y, sr=sr).mean(axis=1))

        return round(tempo), key_mode[0] if key_mode else "N/A"
    except Exception as e:
        print(f"ERROR DE ANÁLISIS DE AUDIO: Falló el análisis de '{filepath}'. Mensaje: {e}")
        print("POSIBLE SOLUCIÓN: Asegúrate de que el archivo no está corrupto o es de un formato soportado por Librosa (MP3, WAV, FLAC).")
        return "N/A", "N/A"

def update_audio_metadata(filepath, title, artist, genre, bpm, key):
    """
    Actualiza los metadatos (ID3 tags) de un archivo MP3.
    Esta función solo actualiza el archivo LOCAL. El script no lo re-sube a Drive.
    """
    if not filepath or not filepath.lower().endswith('.mp3'):
        return

    try:
        audio = MP3(filepath, ID3=ID3)
        if audio.tags is None:
            audio.add_tags()

        # Añadir o actualizar tags ID3 estándar
        audio.tags.add(TIT2(encoding=3, text=[title])) # Título
        audio.tags.add(TPE1(encoding=3, text=[artist])) # Artista
        audio.tags.add(TCON(encoding=3, text=[genre])) # Género

        # Usar TXXX (campo de texto personalizado) para BPM y Clave
        audio.tags.delall('TXXX') # Eliminar tags TXXX existentes para evitar duplicados
        audio.tags.add(TXXX(encoding=3, desc='BPM', text=[str(bpm)]))
        audio.tags.add(TXXX(encoding=3, desc='Key', text=[key]))

        audio.save()
        # print(f"Metadatos actualizados para {filepath} localmente.") # Comentado para minimizar output
    except ID3NoHeaderError:
        print(f"ADVERTENCIA: No se encontraron tags ID3 válidos en '{filepath}'. No se pudieron actualizar los metadatos.")
    except Exception as e:
        print(f"ERROR: Falló la actualización de metadatos de '{filepath}'. Mensaje: {e}")

def move_drive_file(drive_service, file_id, old_parent_id, new_parent_id):
    """Mueve un archivo de Google Drive de una carpeta a otra. Maneja errores de la API de Drive."""
    try:
        file_metadata = drive_service.files().get(fileId=file_id, fields='parents,name').execute()
        current_parents = file_metadata.get('parents', [])

        if old_parent_id not in current_parents:
            print(f"ADVERTENCIA: Archivo '{file_metadata.get('name', 'N/A')}' (ID: {file_id}) NO se encontró en la carpeta de origen '{old_parent_id}'.")
            print("POSIBLE SOLUCIÓN: El archivo ya fue movido manualmente, eliminado, o el ID de la carpeta de origen es incorrecto. No se intentará mover.")
            return False

        drive_service.files().update(
            fileId=file_id,
            addParents=new_parent_id,
            removeParents=old_parent_id,
            fields='id, parents'
        ).execute()
        # print(f"Archivo {file_id} movido exitosamente en Google Drive.") # Comentado para minimizar output
        return True
    except Exception as e:
        print(f"ERROR: Falló el movimiento del archivo '{file_metadata.get('name', 'N/A')}' (ID: {file_id}) en Google Drive. Mensaje: {e}")
        print("POSIBLE SOLUCIÓN: Problema de permisos de la cuenta de servicio en la carpeta de destino, ID de carpeta incorrecto, o error temporal de Google Drive.")
        return False

def main():
    print("\n--- INICIANDO PROCESO DE ORGANIZACIÓN DE BEATS ---")
    gc, drive_service, creds = authenticate_google()
    if gc is None:
        print("SCRIPT DETENIDO: No se pudo autenticar con Google.")
        return

    spreadsheet_id_from_config = read_spreadsheet_id_from_config() # Leer el ID desde el archivo de configuración

    # Pasamos el ID leído del config al intentar obtener la hoja
    worksheet, records, headers = get_sheet_data(gc, spreadsheet_id_from_config, WORKSHEET_NAME)
    if worksheet is None:
        print("SCRIPT DETENIDO: No se pudo acceder/crear la hoja de cálculo.")
        return

    # Verificar que las columnas requeridas existan en la hoja
    required_cols = ['Nombre del Archivo Original', 'Género', 'Ruta en Drive (ID)', 'Enlace de Google Drive', 'BPM', 'Clave Armónica', 'Estado (PENDIENTE/ORGANIZADO)']
    if not all(col in headers for col in required_cols):
        print("ERROR CRÍTICO: Faltan una o más columnas requeridas en tu Google Sheet o los nombres no coinciden.")
        print("VERIFICAR: La PRIMERA FILA de tu Google Sheet debe tener estas columnas (nombres EXACTOS, incluyendo mayúsculas/minúsculas y espacios):")
        for col in required_cols:
            print(f"- '{col}'")
        print("SCRIPT DETENIDO: Por favor, corrige los nombres de las columnas en tu Google Sheet.")
        return

    print("\n--- Paso 1: Actualizando Catálogo de Beats en Google Sheet ---")

    drive_files_in_new_folder = get_drive_file_info(drive_service, NUEVOS_BEATS_FOLDER_ID)
    if not drive_files_in_new_folder and drive_files_in_new_folder != {}: # Si hubo un error en get_drive_file_info y no es un diccionario vacío
        print("SCRIPT DETENIDO: No se pudo obtener información de archivos en la carpeta 'Nuevos Beats'.")
        return

    existing_sheet_names = {rec.get('Nombre del Archivo Original') for rec in records if rec.get('Nombre del Archivo Original')} # Filtrar None o vacíos

    new_beats_added_to_sheet = 0
    for file_name, info in drive_files_in_new_folder.items():
        if file_name not in existing_sheet_names:
            row_data = {
                'Nombre del Archivo Original': file_name,
                'Género': '',
                'Ruta en Drive (ID)': info['id'],
                'Enlace de Google Drive': f"https://drive.google.com/file/d/{info['id']}/view",
                'BPM': '',
                'Clave Armónica': '',
                'Estado (PENDIENTE/ORGANIZADO)': 'PENDIENTE_ANALISIS_Y_MOVIMIENTO'
            }
            new_row_values = [row_data.get(header, '') for header in headers]
            try:
                worksheet.append_row(new_row_values)
                records.append(row_data) # Actualizar records localmente
                new_beats_added_to_sheet += 1
                print(f" -> Añadido a la hoja: {file_name}")
            except Exception as e:
                print(f"ERROR: No se pudo añadir la fila para '{file_name}' a la hoja. Mensaje: {e}")
                print("POSIBLE SOLUCIÓN: Problema de escritura en Google Sheet o cuota API.")

    if new_beats_added_to_sheet > 0:
        print(f"\n--- Se han añadido {new_beats_added_to_sheet} nuevos beats a la hoja. ---")
        print("ACCIÓN REQUERIDA: Abre tu Google Sheet y ASIGNA UN GÉNERO a los beats con estado 'PENDIENTE_ANALISIS_Y_MOVIMIENTO' o 'ANALIZADO_PENDIENTE_GENERO' (columna 'Género').")
        # Recargar datos de la hoja para asegurar que los registros recién añadidos están presentes para el siguiente paso
        try:
            worksheet = gc.open_by_key(worksheet.spreadsheet.id).worksheet(WORKSHEET_NAME)
            records = worksheet.get_all_records()
            headers = worksheet.row_values(1)
        except gspread.exceptions.WorksheetNotFound:
            print(f"ADVERTENCIA: La pestaña '{WORKSHEET_NAME}' no se encontró al recargar. Usando la primera.")
            worksheet = gc.open_by_key(worksheet.spreadsheet.id).get_worksheet(0)
            records = worksheet.get_all_records()
            headers = worksheet.row_values(1)
        except Exception as e:
            print(f"ERROR: Falló la recarga de datos de la hoja después de añadir nuevos beats. Mensaje: {e}")
            print("SCRIPT DETENIDO: Por favor, revisa la hoja y ejecuta de nuevo.")
            return

    print("\n--- Paso 2: Analizando y Moviendo Beats ---")
    processed_count = 0
    for i, record in enumerate(records):
        row_num = i + 2
        file_name = record.get('Nombre del Archivo Original')
        drive_file_id = record.get('Ruta en Drive (ID)')
        genre = record.get('Género', '').strip()
        status = record.get('Estado (PENDIENTE/ORGANIZADO)')

        if not file_name or not drive_file_id:
            continue

        if (status == 'PENDIENTE_ANALISIS_Y_MOVIMIENTO' or (not record.get('BPM') or not record.get('Clave Armónica')) or \
           status == 'ERROR_DESCARGA' or status == 'ERROR_ANALISIS') and status != 'ORGANIZADO':
            if status == 'ORGANIZADO': continue

            print(f" -> Analizando beat: {file_name}...")
            local_filepath = None
            try:
                local_filepath = download_file(drive_service, drive_file_id, file_name, DOWNLOAD_TEMP_DIR, creds)
                if local_filepath:
                    bpm, key = analyze_audio(local_filepath)
                    update_sheet_row(worksheet, row_num, 'BPM', bpm, headers)
                    update_sheet_row(worksheet, row_num, 'Clave Armónica', key, headers)
                    print(f"    - Análisis completado: BPM={bpm}, Clave={key}")
                    if not genre:
                        update_sheet_row(worksheet, row_num, 'Estado (PENDIENTE/ORGANIZADO)', 'ANALIZADO_PENDIENTE_GENERO', headers)
                    else:
                        update_sheet_row(worksheet, row_num, 'Estado (PENDIENTE/ORGANIZADO)', 'PENDIENTE_MOVIMIENTO', headers)
                else:
                    update_sheet_row(worksheet, row_num, 'Estado (PENDIENTE/ORGANIZADO)', 'ERROR_DESCARGA', headers)
            except Exception as e:
                print(f"    - ERROR FATAL al analizar/descargar '{file_name}'. Mensaje: {e}")
                update_sheet_row(worksheet, row_num, 'Estado (PENDIENTE/ORGANIZADO)', 'ERROR_ANALISIS', headers)
            finally:
                if local_filepath and os.path.exists(local_filepath):
                    try: os.remove(local_filepath)
                    except OSError as e: print(f"ADVERTENCIA: No se pudo eliminar archivo temporal '{local_filepath}'. Mensaje: {e}")
            continue

        if genre and (status == 'ANALIZADO_PENDIENTE_GENERO' or status == 'PENDIENTE_MOVIMIENTO' or \
                      status == 'ERROR_MOVIMIENTO' or status == 'ERROR_MOVIMIENTO_CARPETA' or status == 'ERROR_GENERAL_MOVIMIENTO'):
            if status == 'ORGANIZADO': continue

            print(f" -> Moviendo beat: {file_name} (Género: {genre})...")
            try:
                query_folder = f"name = '{genre}' and mimeType = 'application/vnd.google-apps.folder' and '{ORGANIZED_BEATS_PARENT_FOLDER_ID}' in parents and trashed = false"
                results_folder = drive_service.files().list(q=query_folder, fields="files(id, name)").execute()
                items_folder = results_folder.get('files', [])

                target_genre_folder_id = None
                if items_folder:
                    target_genre_folder_id = items_folder[0]['id']
                else:
                    file_metadata = {'name': genre, 'mimeType': 'application/vnd.google-apps.folder', 'parents': [ORGANIZED_BEATS_PARENT_FOLDER_ID]}
                    new_folder = drive_service.files().create(body=file_metadata, fields='id').execute()
                    target_genre_folder_id = new_folder.get('id')
                    print(f"    - Carpeta de género '{genre}' creada en Drive con ID: {target_genre_folder_id}.")

                if target_genre_folder_id:
                    success = move_drive_file(drive_service, drive_file_id, NUEVOS_BEATS_FOLDER_ID, target_genre_folder_id)
                    if success:
                        update_sheet_row(worksheet, row_num, 'Estado (PENDIENTE/ORGANIZADO)', 'ORGANIZADO', headers)
                        print(f"    - Movido exitosamente a '{genre}' en Drive.")
                        processed_count += 1
                    else:
                        update_sheet_row(worksheet, row_num, 'Estado (PENDIENTE/ORGANIZADO)', 'ERROR_MOVIMIENTO', headers)
                else:
                    update_sheet_row(worksheet, row_num, 'Estado (PENDIENTE/ORGANIZADO)', 'ERROR_MOVIMIENTO_CARPETA', headers)
            except Exception as e:
                print(f"    - ERROR FATAL al mover '{file_name}'. Mensaje: {e}")
                update_sheet_row(worksheet, row_num, 'Estado (PENDIENTE/ORGANIZADO)', 'ERROR_GENERAL_MOVIMIENTO', headers)
        elif status == 'ORGANIZADO':
            pass
        else:
            pass

    print(f"\n--- Proceso Completado. Beats procesados en esta ejecución: {processed_count} ---")
    print("VERIFICAR: Revisa la terminal para ver si hubo errores y tu Google Sheet para el estado de los beats.")

if __name__ == "__main__":
    main()


--- INICIANDO PROCESO DE ORGANIZACIÓN DE BEATS ---
ERROR CRÍTICO DE AUTENTICACIÓN: El archivo de credenciales 'braided-grammar-465202-t4-f058b52a6154.json' NO SE ENCONTRÓ.
POSIBLE SOLUCIÓN: 
  1. Verifica que el nombre del archivo en 'SERVICE_ACCOUNT_FILE' es EXACTO (incluyendo .json).
  2. Asegúrate de que el archivo JSON está en la MISMA CARPETA que este script.
  3. Si lo guardaste en otro lugar, pon la RUTA COMPLETA al archivo.
SCRIPT DETENIDO: No se pudo autenticar con Google.


In [3]:
# @title Configure Gemini API key

import google.generativeai as genai
from google.colab import userdata

gemini_api_secret_name = 'GOOGLE_API_KEY'  # @param {type: "string"}

try:
  GOOGLE_API_KEY=userdata.get(gemini_api_secret_name)
  genai.configure(api_key=GOOGLE_API_KEY)
except userdata.SecretNotFoundError as e:
   print(f'Secret not found\n\nThis expects you to create a secret named {gemini_api_secret_name} in Colab\n\nVisit https://aistudio.google.com/app/apikey to create an API key\n\nStore that in the secrets section on the left side of the notebook (key icon)\n\nName the secret {gemini_api_secret_name}')
   raise e
except userdata.NotebookAccessError as e:
  print(f'You need to grant this notebook access to the {gemini_api_secret_name} secret in order for the notebook to access Gemini on your behalf.')
  raise e
except Exception as e:
  print(f"There was an unknown error. Ensure you have a secret {gemini_api_secret_name} stored in Colab and it's a valid key from https://aistudio.google.com/app/apikey")
  raise e

In [4]:
!pip install matplotlib-venn

In [5]:
# https://pypi.python.org/pypi/pydot
!apt-get -qq install -y graphviz && pip install pydot
import pydot

In [6]:
!apt-get -qq install -y libfluidsynth1

E: Package 'libfluidsynth1' has no installation candidate


In [8]:
from google.colab import userdata
userdata.get('secretName')

'braided-grammar-465202-t4-f058b52a6154'

In [9]:
%pip install mutagen

# Nueva sección

Add `%load_ext cudf.pandas` before importing pandas to speed up operations using GPU

In [10]:
# @title Create a prompt

import google.generativeai as genai
from google.colab import userdata

api_key_name = 'GOOGLE_API_KEY' # @param {type: "string"}
prompt = 'ANALIZE KEY AND BPM THEN ADD TO SHEET AND FILE METADATA IF NECESARY' # @param {type: "string"}
system_instructions = 'CONSISTENT RESULTS' # @param {type: "string"}
model = 'gemini-2.0-flash' # @param {type: "string"} ["gemini-1.0-pro", "gemini-1.5-pro", "gemini-1.5-flash", "gemini-2.0-flash"]
temperature = 0.5 # @param {type: "slider", min: 0, max: 2, step: 0.05}
stop_sequence = '' # @param {type: "string"}

if model == 'gemini-1.0-pro' and system_instructions is not None:
  system_instructions = None
  print('\x1b[31m(WARNING: System instructions ignored, gemini-1.0-pro does not support system instructions)\x1b[0m')

if model == 'gemini-1.0-pro' and temperature > 1:
  temperature = 1
  print('\x1b[34m(INFO: Temperature set to 1, gemini-1.0-pro does not support temperature > 1)\x1b[0m')

if system_instructions == '':
  system_instructions = None

api_key = userdata.get(api_key_name)
genai.configure(api_key=api_key)
model = genai.GenerativeModel(model, system_instruction=system_instructions)

# Fix: Ensure stop_sequences is None or a list of non-empty strings
stop_sequences = [stop_sequence] if stop_sequence else None

config = genai.GenerationConfig(temperature=temperature, stop_sequences=stop_sequences)
response = model.generate_content(contents=[prompt], generation_config=config)
response.text

'Okay, I can do that, but I need more information to give you a precise process. Here\'s a breakdown of what\'s involved and what I need from you:\n\n**1. Understanding the Task:**\n\n*   **Key and BPM Analysis:** This involves using software or algorithms to determine the musical key (e.g., C Major, A minor) and the tempo (Beats Per Minute) of an audio file.\n*   **Adding to Sheet:** This implies you have some kind of spreadsheet or database where you want to store the analyzed key and BPM data.  What kind of sheet is it? (Google Sheet, Excel, CSV, etc.)\n*   **File Metadata:** This means embedding the key and BPM information directly into the audio file itself. This is typically done using ID3 tags (for MP3 files) or similar metadata formats for other audio formats.\n*   **If Necessary:** This means you only want to add the metadata if it\'s missing or incorrect.\n\n**2. Information I Need From You:**\n\nTo give you the best instructions, tell me:\n\n*   **Audio File Format(s):** Wha

In [11]:
from google.colab import userdata
userdata.get('secretName')

'braided-grammar-465202-t4-f058b52a6154'

In [ ]:
%load_ext cudf.pandas
import pandas as pd
import numpy as np

# Randomly generated dataset of parking violations-
# Define the number of rows
num_rows = 1000000

states = ["NY", "NJ", "CA", "TX"]
violations = ["Double Parking", "Expired Meter", "No Parking",
              "Fire Hydrant", "Bus Stop"]
vehicle_types = ["SUBN", "SDN"]

# Create a date range
start_date = "2022-01-01"
end_date = "2022-12-31"
dates = pd.date_range(start=start_date, end=end_date, freq='D')

# Generate random data
data = {
    "Registration State": np.random.choice(states, size=num_rows),
    "Violation Description": np.random.choice(violations, size=num_rows),
    "Vehicle Body Type": np.random.choice(vehicle_types, size=num_rows),
    "Issue Date": np.random.choice(dates, size=num_rows),
    "Ticket Number": np.random.randint(1000000000, 9999999999, size=num_rows)
}

# Create a DataFrame
df = pd.DataFrame(data)

# Which parking violation is most commonly committed by vehicles from various U.S states?

(df[["Registration State", "Violation Description"]]  # get only these two columns
 .value_counts()  # get the count of offences per state and per type of offence
 .groupby("Registration State")  # group by state
 .head(1)  # get the first row in each group (the type of offence with the largest count)
 .sort_index()  # sort by state name
 .reset_index()
)

In [13]:
%pip install mutagen